<a href="https://colab.research.google.com/github/ShauryaK0303/TASK-1-SPAM-CLASSIFICATION/blob/main/Copy_of_Rs_Lab2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix_filled = user_item_matrix.fillna(0)

In [ ]:
cosine_sim = cosine_similarity(user_item_matrix_filled)
cosine_sim_df = pd.DataFrame(cosine_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

In [ ]:
euclidean_sim = np.zeros_like(cosine_sim)
for i in range(len(user_item_matrix.index)):
    for j in range(len(user_item_matrix.index)):
        euclidean_sim[i, j] = 1 / (1 + euclidean(user_item_matrix_filled.iloc[i], user_item_matrix_filled.iloc[j]))
euclidean_sim_df = pd.DataFrame(euclidean_sim, index=user_item_matrix.index, columns=user_item_matrix.index)


In [ ]:
pearson_sim = np.zeros_like(cosine_sim)
for i in range(len(user_item_matrix.index)):
    for j in range(len(user_item_matrix.index)):
        if i != j:
            corr, _ = pearsonr(user_item_matrix_filled.iloc[i], user_item_matrix_filled.iloc[j])
            pearson_sim[i, j] = corr
        else:
            pearson_sim[i, j] = 1
pearson_sim_df = pd.DataFrame(pearson_sim, index=user_item_matrix.index, columns=user_item_matrix.index)


In [ ]:
weights = {'cosine': 0.4, 'euclidean': 0.3, 'pearson': 0.3}
hybrid_sim = (
    weights['cosine'] * cosine_sim +
    weights['euclidean'] * euclidean_sim +
    weights['pearson'] * pearson_sim
)
hybrid_sim_df = pd.DataFrame(hybrid_sim, index=user_item_matrix.index, columns=user_item_matrix.index)


In [ ]:
user_predicted_ratings = pd.DataFrame(index=user_item_matrix.index, columns=user_item_matrix.columns)
for user in user_item_matrix.index:
    sim_scores = hybrid_sim_df[user]
    weighted_sum = sim_scores.values @ user_item_matrix_filled
    sim_sum = np.abs(sim_scores).sum()
    user_predicted_ratings.loc[user] = weighted_sum / sim_sum

In [ ]:
def recommend_items(user_id, num_recommendations=10):
    user_ratings = user_predicted_ratings.loc[user_id].sort_values(ascending=False)
    already_rated = user_item_matrix.loc[user_id].dropna().index
    recommendations = user_ratings.drop(already_rated)
    return recommendations.head(num_recommendations)


In [ ]:
print(recommend_items(1))


movieId
318     2.592568
589     2.033458
858     1.802514
2762     1.61039
4993    1.572033
150     1.549661
32      1.538333
7153    1.452908
588     1.435471
5952    1.434228
Name: 1, dtype: object


In [ ]:

print(recommend_items(2))

movieId
356     2.588259
2571    2.585861
2959    2.434292
296     2.315085
593     2.091803
7153    2.004482
4993     1.99546
260     1.867594
5952    1.832427
50      1.708289
Name: 2, dtype: object


#**ANALYSIS & CONCLUSION**
User-User Collaborative Filtering is effective in generating recommendations by leveraging the preferences of similar users. In part (a), the implementation demonstrated that identifying similar users using methods like Cosine Similarity, Euclidean Distance, and Pearson Correlation helps make personalized recommendations. Each similarity measure has unique characteristics:

* Cosine Similarity captures the angle between preference vectors, focusing on direction rather than magnitude.
* Euclidean Distance emphasizes absolute differences, making it sensitive to scale.
* Pearson Correlation measures linear relationships, accounting for variations in rating scales.

User-User Collaborative Filtering is a practical approach for generating personalized recommendations, with similarity measures like Cosine Similarity and Pearson Correlation proving effective.